# Solution 1: free form chat before creation

In [9]:
from openai import OpenAI

SYSTEM_PROMPT = """
You are a terraform developer, with focus on AWS cloud.
When the user requests to create an AWS resource, you will translate his request to terraform infrastructure as code.
First, after receiving the user request, think about what parameters you need in order to create the resources, like resource names, regions,...
Then ask very brief questions about each of the needed parameters, giving a brief explanation of the parameter. Ask about one parameter at a time.
Once you believe you have all the configuration that you need, give your final answer with terraform infrastructure as code.
Present all the resources under one file. Format the file like this:
```terraform
<your terraform code here>
```
Also present your remarks to the user, explicitly in this format:
```remarks
<your remarks to the user here>
```
Make sure that your terraform code handles all the dependencies to create the requested resource.
"""



client = OpenAI()

request = "create resources suitable for hosting a static website on s3 bucket with route53"

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": request,
        }
    ],
    model="gpt-4o",
)

chat_completion.choices[0].message.content


"To create resources suitable for hosting a static website on S3 with Route 53, we need a few parameters. Let's go through these one by one:\n\n1. **Domain Name**: What is the domain name you would like to use for your static website? This is necessary to configure Route 53 and the bucket.\n\n2. **AWS Region**: Which AWS region would you like to create the S3 bucket in? This is important because S3 buckets are region-specific, although Route 53 is global.\n\n3. **Bucket Policy**: Would you like the S3 bucket to be publicly accessible? This is required for a static website so that visitors can access your content.\n\n4. **Index Document**: What is the main file of your website (usually `index.html`) that should be used as the index document for the S3 bucket?\n\nPlease provide this information so I can continue creating the resources."

Problems:
chat is verbose
The assistant can ask about many paramters at once

# Solution 2: one-shot pre generation parameter creation

In [23]:
from openai import OpenAI
from enum import Enum
from typing import Union

from pydantic import BaseModel

client = OpenAI()

class ParameterType(str, Enum):
    advanced = "advanced"
    basic = "basic"

class Parameter(BaseModel):
    parameter_name: str
    parameter_description: str
    default_value: str
    parameter_type: ParameterType
    
class ParamtersResponse(BaseModel):
    parameters: list[Parameter]


SYSTEM_PROMPT = """
The user will request you to create resources on AWS by generating terraform infrastructure as code. 
Generate a list parameters you need to create those resources. Each parameter consists of parameter name, parameter description and default value that you suggest to the user.
"""

request = "create resources suitable for hosting a static website on s3 bucket with route53"

completion = client.beta.chat.completions.parse(
  model="gpt-4o-2024-08-06",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": SYSTEM_PROMPT
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": request
        }
      ]
    },
  ],
  temperature=1,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format=ParamtersResponse,
)
completion.choices[0].message.parsed

ParamtersResponse(parameters=[Parameter(parameter_name='bucket_name', parameter_description='The name of the S3 bucket to host the static website.', default_value='my-static-website', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='region', parameter_description='The AWS region where the S3 bucket will be created.', default_value='us-east-1', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='index_document', parameter_description='The name of the index document for the S3 bucket hosting the website.', default_value='index.html', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='error_document', parameter_description='The name of the error document for the S3 bucket hosting the website.', default_value='error.html', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='route53_domain_name', parameter_description='The domain name to be configured in Route53 for the static website.', default_valu

In [24]:
for parameter in completion.choices[0].message.parsed.parameters:
    print(parameter.json())

{"parameter_name":"bucket_name","parameter_description":"The name of the S3 bucket to host the static website.","default_value":"my-static-website","parameter_type":"basic"}
{"parameter_name":"region","parameter_description":"The AWS region where the S3 bucket will be created.","default_value":"us-east-1","parameter_type":"basic"}
{"parameter_name":"index_document","parameter_description":"The name of the index document for the S3 bucket hosting the website.","default_value":"index.html","parameter_type":"basic"}
{"parameter_name":"error_document","parameter_description":"The name of the error document for the S3 bucket hosting the website.","default_value":"error.html","parameter_type":"basic"}
{"parameter_name":"route53_domain_name","parameter_description":"The domain name to be configured in Route53 for the static website.","default_value":"example.com","parameter_type":"basic"}
{"parameter_name":"route53_zone_id","parameter_description":"The ID of the Route53 hosted zone that will 

problems:
choosing one parameter can change the course of the paramters to be chosen, for instance, if user chooses a certain type of route53_record, he may need specific parameters for that record type.

# Solution 3: multi-step pre generation parameter creation

In [22]:
from openai import OpenAI
from enum import Enum
from typing import Union, List, Optional

from pydantic import BaseModel

client = OpenAI()

class Parameter(BaseModel):
    parameter_name: str
    parameter_description: str
    default_value: str

class Response(BaseModel):
    parameter: Optional[Parameter]
    finished: bool



SYSTEM_PROMPT = """
The user will request you to create resources on AWS by generating terraform infrastructure as code. 
Generate a list parameters you need to create those resources. Each parameter consists of parameter name, parameter description and default value that you suggest to the user.
Here are the parameters that the user provided so far:
{parameters}

Suggest one parameter at a time if you need more parameters and set finished=False.
If you don't need anymore parameters, don't return any parameter and return finished=True
"""

def fmt_parameters(params: List[Parameter]):
    if not params:
        return "No parameters provided so far."
    params = [f"{parameter.parameter_name} ({parameter.parameter_description}): {parameter.default_value}" for parameter in params]
    return "\n".join(params)


params = []

while True:
    request = "create resources suitable for hosting a static website on s3 bucket with route53"
    
    completion = client.beta.chat.completions.parse(
      model="gpt-4o-2024-08-06",
      messages=[
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": SYSTEM_PROMPT.format(parameters=fmt_parameters(params))
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": request
            }
          ]
        },
      ],
      temperature=1,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      response_format=Response,
    )
    response = completion.choices[0].message.parsed
    if response.finished:
        break
    else:
        param = response.parameter
        assert param is not None
        print(param)
        param.default_value = input("provide a value: ")
        params.append(param)

parameter_name='bucket_name' parameter_description='The name of the S3 bucket where the static website will be hosted. It must be globally unique.' default_value='my-static-website-bucket'


provide a value:  my-static-website-bucket


parameter_name='domain_name' parameter_description='The registered domain name to be used for the static website.' default_value='example.com'


provide a value:  example.com


parameter_name='region' parameter_description='The AWS region where the S3 bucket will be created.' default_value='us-east-1'


...

provide a value:  


parameter_name='bucket_acl' parameter_description="The canned ACL to apply to the S3 bucket to set its permissions. Possible values include 'private', 'public-read', etc." default_value='public-read'


provide a value:  


parameter_name='enable_website_redirect' parameter_description='Specifies whether the S3 bucket should configure a redirect for requests. Useful for redirecting from the bare domain to www or vice versa.' default_value='false'


KeyboardInterrupt: Interrupted by user

problems:
sometimes, asks about a resource that is supposed to be created. The assistant should rather create this resource for the user
for instance
parameter_name='certificate_arn' parameter_description='The ARN of the SSL certificate in AWS Certificate Manager (ACM) to be used if HTTPS is enabled for the static website.' default_value='arn:aws:acm:region:account-id:certificate/certificate-id'
too many parameters and no way to mask out advanced parameters

# Solution 4: one-shot post generation parameter creation with string replacement

In [25]:
from openai import OpenAI

SYSTEM_PROMPT = """
You are a terraform developer, with focus on AWS cloud.
When the user requests to create an AWS resource, translate his request to terraform infrastructure as code.
Present all the resources under one file. Format the file like this:
```terraform
<your terraform code here>
```
Make sure that your terraform code handles all the dependencies to create the requested resource.
"""


client = OpenAI()

request = "create resources suitable for hosting a static website on s3 bucket with route53"

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": request,
        }
    ],
    model="gpt-4o",
)

chat_completion.choices[0].message.content

'To host a static website on AWS using S3 and Route 53, you need to create an S3 bucket configured for static website hosting and a Route 53 DNS record to point to the S3 bucket website endpoint. Below is the Terraform code to achieve this setup:\n\n```terraform\nprovider "aws" {\n  region = "us-east-1"  # You can change this to your preferred region\n}\n\nresource "aws_s3_bucket" "static_website" {\n  bucket = "your-unique-bucket-name"  # Ensure this bucket name is unique globally\n  acl    = "public-read"\n\n  website {\n    index_document = "index.html"\n    error_document = "error.html"\n  }\n}\n\nresource "aws_s3_bucket_policy" "bucket_policy" {\n  bucket = aws_s3_bucket.static_website.id\n\n  policy = jsonencode({\n    Version = "2012-10-17"\n    Statement = [\n      {\n        Effect    = "Allow"\n        Principal = "*"\n        Action    = "s3:GetObject"\n        Resource  = "${aws_s3_bucket.static_website.arn}/*"\n      }\n    ]\n  })\n}\n\nresource "aws_route53_zone" "myzone

In [27]:
import re
from typing import List, Optional

def extract_code_blocks(text: str, title: Optional[str] = None) -> List[str]:
    """
    Extract code blocks from text that match the pattern ```{title}\n<content>```

    Args:
        text (str): The input text containing code blocks
        title (Optional[str]): If provided, only extract blocks with this title

    Returns:
        List[str]: List of extracted code block contents
    """
    # Pattern explanation:
    # ```           - Match opening triple backticks
    # ([^\n]*)     - Capture any characters except newline (the title)
    # \n           - Match newline
    # (.*?)        - Capture any characters (non-greedy) (the content)
    # ```          - Match closing triple backticks
    pattern = r'```([^\n]*)\n(.*?)```'

    # Use re.DOTALL to make dot match newlines
    matches = re.finditer(pattern, text, re.DOTALL)

    results = []
    for match in matches:
        block_title = match.group(1).strip()
        content = match.group(2).strip()

        # If title is specified, only include blocks with matching title
        if title is None or block_title.lower() == title.lower():
            results.append(content)

    return results

In [29]:
terraform_code = extract_code_blocks(chat_completion.choices[0].message.content, "terraform")[0]
terraform_code

'provider "aws" {\n  region = "us-east-1"  # You can change this to your preferred region\n}\n\nresource "aws_s3_bucket" "static_website" {\n  bucket = "your-unique-bucket-name"  # Ensure this bucket name is unique globally\n  acl    = "public-read"\n\n  website {\n    index_document = "index.html"\n    error_document = "error.html"\n  }\n}\n\nresource "aws_s3_bucket_policy" "bucket_policy" {\n  bucket = aws_s3_bucket.static_website.id\n\n  policy = jsonencode({\n    Version = "2012-10-17"\n    Statement = [\n      {\n        Effect    = "Allow"\n        Principal = "*"\n        Action    = "s3:GetObject"\n        Resource  = "${aws_s3_bucket.static_website.arn}/*"\n      }\n    ]\n  })\n}\n\nresource "aws_route53_zone" "myzone" {\n  name = "example.com"  # Replace with your domain name\n}\n\nresource "aws_route53_record" "www" {\n  zone_id = aws_route53_zone.myzone.zone_id\n  name    = "www"\n  type    = "CNAME"\n  ttl     = 300\n  records = [aws_s3_bucket.static_website.website_endpo

In [30]:
from openai import OpenAI
from enum import Enum
from typing import Union

from pydantic import BaseModel

client = OpenAI()

class ParameterType(str, Enum):
    advanced = "advanced"
    basic = "basic"

class Parameter(BaseModel):
    parameter_name: str
    parameter_description: str
    default_value: str
    parameter_type: ParameterType

class Response(BaseModel):
    parameters: List[Parameter]


SYSTEM_PROMPT = """
The user provided the following request you to create resources on AWS by generating terraform infrastructure as code.
{request}
Given terraform code, generate a list parameters you need to customize those resources. Each parameter consists of parameter name, parameter description, default value and parameter type that you will to the user to customize the resources.
"""

request = "create resources suitable for hosting a static website on s3 bucket with route53"

completion = client.beta.chat.completions.parse(
  model="gpt-4o-2024-08-06",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": SYSTEM_PROMPT.format(request=request)
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"here is the terraform code \n {terraform_code}"
        }
      ]
    },
  ],
  temperature=1,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format=ParamtersResponse,
)
completion.choices[0].message.parsed

ParamtersResponse(parameters=[Parameter(parameter_name='aws_region', parameter_description='The AWS region where resources will be created.', default_value='us-east-1', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='s3_bucket_name', parameter_description='A unique name for the S3 bucket, must be globally unique.', default_value='your-unique-bucket-name', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='s3_index_document', parameter_description='The path to the index document for the S3 website.', default_value='index.html', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='s3_error_document', parameter_description='The path to the error document for the S3 website.', default_value='error.html', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='route53_domain_name', parameter_description='The domain name for the Route 53 hosted zone.', default_value='example.com', parameter_type=<Paramete

In [31]:
terraform_code

'provider "aws" {\n  region = "us-east-1"  # You can change this to your preferred region\n}\n\nresource "aws_s3_bucket" "static_website" {\n  bucket = "your-unique-bucket-name"  # Ensure this bucket name is unique globally\n  acl    = "public-read"\n\n  website {\n    index_document = "index.html"\n    error_document = "error.html"\n  }\n}\n\nresource "aws_s3_bucket_policy" "bucket_policy" {\n  bucket = aws_s3_bucket.static_website.id\n\n  policy = jsonencode({\n    Version = "2012-10-17"\n    Statement = [\n      {\n        Effect    = "Allow"\n        Principal = "*"\n        Action    = "s3:GetObject"\n        Resource  = "${aws_s3_bucket.static_website.arn}/*"\n      }\n    ]\n  })\n}\n\nresource "aws_route53_zone" "myzone" {\n  name = "example.com"  # Replace with your domain name\n}\n\nresource "aws_route53_record" "www" {\n  zone_id = aws_route53_zone.myzone.zone_id\n  name    = "www"\n  type    = "CNAME"\n  ttl     = 300\n  records = [aws_s3_bucket.static_website.website_endpo

# Solution 5: 2-shot generation with parameter creation

In [32]:
from openai import OpenAI

SYSTEM_PROMPT = """
You are a terraform developer, with focus on AWS cloud.
When the user requests to create an AWS resource, translate his request to terraform infrastructure as code.
Present all the resources under one file. Format the file like this:
```terraform
<your terraform code here>
```
Make sure that your terraform code handles all the dependencies to create the requested resource.
"""


client = OpenAI()

request = "create resources suitable for hosting a static website on s3 bucket with route53"

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": request,
        }
    ],
    model="gpt-4o",
)

chat_completion.choices[0].message.content

'To host a static website using an S3 bucket with a custom domain managed by Route 53, you\'ll need to set up the S3 bucket with website hosting enabled, configure Route 53 to route traffic to the bucket, and create an SSL certificate for HTTPS support. Assuming you already have a registered domain in Route 53, here\'s the necessary Terraform code:\n\n```terraform\nprovider "aws" {\n  region = "us-east-1"\n}\n\nvariable "domain_name" {\n  description = "The domain name to be used for the static website"\n}\n\nresource "aws_s3_bucket" "website" {\n  bucket = var.domain_name\n  acl    = "public-read"\n\n  website {\n    index_document = "index.html"\n    error_document = "error.html"\n  }\n}\n\nresource "aws_s3_bucket_policy" "website_policy" {\n  bucket = aws_s3_bucket.website.id\n  policy = jsonencode({\n    Version = "2012-10-17"\n    Statement = [\n      {\n        Effect    = "Allow"\n        Principal = "*"\n        Action    = "s3:GetObject"\n        Resource  = "${aws_s3_bucket.w

In [33]:
import re
from typing import List, Optional

def extract_code_blocks(text: str, title: Optional[str] = None) -> List[str]:
    """
    Extract code blocks from text that match the pattern ```{title}\n<content>```

    Args:
        text (str): The input text containing code blocks
        title (Optional[str]): If provided, only extract blocks with this title

    Returns:
        List[str]: List of extracted code block contents
    """
    # Pattern explanation:
    # ```           - Match opening triple backticks
    # ([^\n]*)     - Capture any characters except newline (the title)
    # \n           - Match newline
    # (.*?)        - Capture any characters (non-greedy) (the content)
    # ```          - Match closing triple backticks
    pattern = r'```([^\n]*)\n(.*?)```'

    # Use re.DOTALL to make dot match newlines
    matches = re.finditer(pattern, text, re.DOTALL)

    results = []
    for match in matches:
        block_title = match.group(1).strip()
        content = match.group(2).strip()

        # If title is specified, only include blocks with matching title
        if title is None or block_title.lower() == title.lower():
            results.append(content)

    return results

In [34]:
terraform_code = extract_code_blocks(chat_completion.choices[0].message.content, "terraform")[0]
terraform_code

'provider "aws" {\n  region = "us-east-1"\n}\n\nvariable "domain_name" {\n  description = "The domain name to be used for the static website"\n}\n\nresource "aws_s3_bucket" "website" {\n  bucket = var.domain_name\n  acl    = "public-read"\n\n  website {\n    index_document = "index.html"\n    error_document = "error.html"\n  }\n}\n\nresource "aws_s3_bucket_policy" "website_policy" {\n  bucket = aws_s3_bucket.website.id\n  policy = jsonencode({\n    Version = "2012-10-17"\n    Statement = [\n      {\n        Effect    = "Allow"\n        Principal = "*"\n        Action    = "s3:GetObject"\n        Resource  = "${aws_s3_bucket.website.arn}/*"\n      }\n    ]\n  })\n}\n\nresource "aws_route53_record" "www" {\n  zone_id = aws_route53_zone.primary.zone_id\n  name    = var.domain_name\n  type    = "A"\n\n  alias {\n    name                   = aws_cloudfront_distribution.website.domain_name\n    zone_id                = aws_cloudfront_distribution.website.hosted_zone_id\n    evaluate_target_h

In [36]:
from openai import OpenAI
from enum import Enum
from typing import Union

from pydantic import BaseModel

client = OpenAI()

class ParameterType(str, Enum):
    advanced = "advanced"
    basic = "basic"

class Parameter(BaseModel):
    parameter_name: str
    parameter_description: str
    default_value: str
    parameter_type: ParameterType

class Response(BaseModel):
    parameters: List[Parameter]


SYSTEM_PROMPT = """
The user provided the following request you to create resources on AWS by generating terraform infrastructure as code.
{request}
Given terraform code, generate a list parameters you need to customize those resources. Each parameter consists of parameter name, parameter description, default value and parameter type that you will to the user to customize the resources.
"""

request = "create resources suitable for hosting a static website on s3 bucket with route53"

completion = client.beta.chat.completions.parse(
  model="gpt-4o-2024-08-06",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": SYSTEM_PROMPT.format(request=request)
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"here is the terraform code \n {terraform_code}"
        }
      ]
    },
  ],
  temperature=1,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format=ParamtersResponse,
)
completion.choices[0].message.parsed

ParamtersResponse(parameters=[Parameter(parameter_name='aws_region', parameter_description='The AWS region where the resources will be created', default_value='us-east-1', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='domain_name', parameter_description='The domain name to be used for the static website', default_value='', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='website_index_document', parameter_description="The S3 bucket's index document for the website", default_value='index.html', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='website_error_document', parameter_description="The S3 bucket's error document for the website", default_value='error.html', parameter_type=<ParameterType.basic: 'basic'>), Parameter(parameter_name='certificate_validation_method', parameter_description='The method for validating the certificate', default_value='DNS', parameter_type=<ParameterType.advanced: 'advanced'>), P

In [39]:
parameters = completion.choices[0].message.parsed.parameters
values = ["eu-central-1", "test.com", "index-success.html", "error.html", "DNS", "PriceClass_100"]
for param, value in zip(parameters, values):
    param.default_value = value
parameters

[Parameter(parameter_name='aws_region', parameter_description='The AWS region where the resources will be created', default_value='eu-central-1', parameter_type=<ParameterType.basic: 'basic'>),
 Parameter(parameter_name='domain_name', parameter_description='The domain name to be used for the static website', default_value='test.com', parameter_type=<ParameterType.basic: 'basic'>),
 Parameter(parameter_name='website_index_document', parameter_description="The S3 bucket's index document for the website", default_value='index-success.html', parameter_type=<ParameterType.basic: 'basic'>),
 Parameter(parameter_name='website_error_document', parameter_description="The S3 bucket's error document for the website", default_value='error.html', parameter_type=<ParameterType.basic: 'basic'>),
 Parameter(parameter_name='certificate_validation_method', parameter_description='The method for validating the certificate', default_value='DNS', parameter_type=<ParameterType.advanced: 'advanced'>),
 Parame

In [40]:
def fmt_parameters(params: List[Parameter]):
    if not params:
        return "No parameters provided so far."
    params = [f"{parameter.parameter_name} ({parameter.parameter_description}): {parameter.default_value}" for parameter in params]
    return "\n".join(params)

from openai import OpenAI

SYSTEM_PROMPT = """
You are a terraform developer, with focus on AWS cloud.
When the user requests to create an AWS resource, translate his request to terraform infrastructure as code.
Present all the resources under one file. Format the file like this:
```terraform
<your terraform code here>
```
Also present your remarks to the user, explicitly in this format:
```remarks
<your remarks to the user here>
```
Make sure that your terraform code handles all the dependencies to create the requested resource.
"""


client = OpenAI()

request = "create resources suitable for hosting a static website on s3 bucket with route53"
USER_PROMPT = """
{request}
Use the folliowing configuration:
{configuration}
"""
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": USER_PROMPT.format(request=request, configuration=fmt_parameters(parameters)),
        }
    ],
    model="gpt-4o",
)

chat_completion.choices[0].message.content

'```terraform\nprovider "aws" {\n  region = "eu-central-1"\n}\n\nresource "aws_s3_bucket" "website_bucket" {\n  bucket = "test.com"\n  acl    = "public-read"\n\n  website {\n    index_document = "index-success.html"\n    error_document = "error.html"\n  }\n}\n\nresource "aws_s3_bucket_policy" "website_bucket_policy" {\n  bucket = aws_s3_bucket.website_bucket.id\n\n  policy = <<EOF\n{\n  "Version": "2012-10-17",\n  "Statement": [\n    {\n      "Sid": "PublicReadGetObject",\n      "Effect": "Allow",\n      "Principal": "*",\n      "Action": "s3:GetObject",\n      "Resource": "arn:aws:s3:::test.com/*"\n    }\n  ]\n}\nEOF\n}\n\nresource "aws_route53_zone" "example" {\n  name = "test.com"\n}\n\nresource "aws_acm_certificate" "cert" {\n  domain_name               = "test.com"\n  validation_method         = "DNS"\n  \n  lifecycle {\n    create_before_destroy = true\n  }\n}\n\nresource "aws_route53_record" "cert_validation" {\n  for_each = {\n    for dvo in aws_acm_certificate.cert.domain_vali

In [42]:
print(extract_code_blocks(chat_completion.choices[0].message.content, "terraform")[0])

provider "aws" {
  region = "eu-central-1"
}

resource "aws_s3_bucket" "website_bucket" {
  bucket = "test.com"
  acl    = "public-read"

  website {
    index_document = "index-success.html"
    error_document = "error.html"
  }
}

resource "aws_s3_bucket_policy" "website_bucket_policy" {
  bucket = aws_s3_bucket.website_bucket.id

  policy = <<EOF
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "PublicReadGetObject",
      "Effect": "Allow",
      "Principal": "*",
      "Action": "s3:GetObject",
      "Resource": "arn:aws:s3:::test.com/*"
    }
  ]
}
EOF
}

resource "aws_route53_zone" "example" {
  name = "test.com"
}

resource "aws_acm_certificate" "cert" {
  domain_name               = "test.com"
  validation_method         = "DNS"
  
  lifecycle {
    create_before_destroy = true
  }
}

resource "aws_route53_record" "cert_validation" {
  for_each = {
    for dvo in aws_acm_certificate.cert.domain_validation_options : dvo.domain_name => {
      name   = dvo.resour